# Face Mask detector

In [2]:
# Part 1: Import Libraries and Define Paths
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pickle

dataset_path = r"E:\Machine Learning\Face Mask"
model_save_path = r"E:\Machine Learning\face_mask_model"

if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

In [3]:
# Part 2: Data Preprocessing and Augmentation
IMG_SIZE = 64  
BATCH_SIZE = 8
EPOCHS = 10

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    validation_split=0.20
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.20)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 5960 images belonging to 2 classes.
Found 1490 images belonging to 2 classes.


In [5]:
# Part 3: Build and Compile the CNN Model

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
# Part 4: Train the Model and Save Class Indices
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = validation_generator.samples // BATCH_SIZE

print("[INFO] Training model...")
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

class_indices = train_generator.class_indices
class_indices_flipped = {v: k for k, v in class_indices.items()}

with open(os.path.join(model_save_path, "class_indices.pickle"), "wb") as f:
    pickle.dump(class_indices_flipped, f)
    print(f"Class indices saved to {os.path.join(model_save_path, 'class_indices.pickle')}")
    print(f"Class mapping: {class_indices_flipped}")

[INFO] Training model...
Epoch 1/10
270/745 [=========>....................] - ETA: 31s - loss: 0.5837 - accuracy: 0.6981

C:\ProgramData\miniconda3\envs\ducat\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


745/745 [==============================] - 74s 80ms/step - loss: 0.4865 - accuracy: 0.7725 - val_loss: 0.3048 - val_accuracy: 0.8763
Epoch 2/10
745/745 [==============================] - 15s 20ms/step - loss: 0.3885 - accuracy: 0.8349 - val_loss: 0.2319 - val_accuracy: 0.9194
Epoch 3/10
745/745 [==============================] - 15s 20ms/step - loss: 0.3394 - accuracy: 0.8602 - val_loss: 0.1814 - val_accuracy: 0.9368
Epoch 4/10
745/745 [==============================] - 15s 20ms/step - loss: 0.2871 - accuracy: 0.8826 - val_loss: 0.1714 - val_accuracy: 0.9368
Epoch 5/10
745/745 [==============================] - 15s 20ms/step - loss: 0.2471 - accuracy: 0.9042 - val_loss: 0.1538 - val_accuracy: 0.9476
Epoch 6/10
745/745 [==============================] - 15s 20ms/step - loss: 0.2307 - accuracy: 0.9146 - val_loss: 0.1394 - val_accuracy: 0.9476
Epoch 7/10
745/745 [==============================] - 15s 21ms/step - loss: 0.2150 - accuracy: 0.9208 - val_loss: 0.1404 - val_accuracy: 0.9503
Epo

In [7]:
# Part 5: Save Model and Print Accuracy

# Save the model

saved_model_path = os.path.join(model_save_path, "saved_model")
tf.saved_model.save(model, saved_model_path)
print(f"Model saved to {saved_model_path}")

model_json = model.to_json()
with open(os.path.join(model_save_path, "model_config.json"), "w") as json_file:
    json_file.write(model_json)
    print(f"Model config saved to {os.path.join(model_save_path, 'model_config.json')}")

with open(os.path.join(model_save_path, "model_summary.txt"), "w") as summary_file:
    model.summary(print_fn=lambda x: summary_file.write(x + '\n'))
    print(f"Model summary saved to {os.path.join(model_save_path, 'model_summary.txt')}")

# Print final training and validation accuracy
print("[INFO] Model training complete!")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

INFO:tensorflow:Assets written to: E:\Machine Learning\face_mask_model\saved_model\assets


INFO:tensorflow:Assets written to: E:\Machine Learning\face_mask_model\saved_model\assets


Model saved to E:\Machine Learning\face_mask_model\saved_model
Model config saved to E:\Machine Learning\face_mask_model\model_config.json
Model summary saved to E:\Machine Learning\face_mask_model\model_summary.txt
[INFO] Model training complete!
Final Training Accuracy: 0.9305
Final Validation Accuracy: 0.9449


In [9]:
# Part 6: Load Face Detector and Mask Detector Model
import cv2
import numpy as np
import os
import tensorflow as tf
import pickle
import time

face_cascade_path = r"E:\Machine Learning\dataset\haarcascade_frontalface_default.xml"
if os.path.exists(face_cascade_path):
    face_detector = cv2.CascadeClassifier(face_cascade_path)
    print(f"Using Haar Cascade face detector: {face_cascade_path}")
else:
    print("[ERROR] Haar Cascade face detector not found")
    exit()

model_save_path = r"E:\Machine Learning\face_mask_model"
saved_model_path = os.path.join(model_save_path, "saved_model")

print("[INFO] Loading face mask detector model...")
try:
    mask_detector = tf.saved_model.load(saved_model_path)
    print("Model loaded successfully")
    infer = mask_detector.signatures["serving_default"]
    print("Model signature:", list(infer.structured_outputs.keys()))
except Exception as e:
    print(f"[ERROR] Failed to load model: {e}")
    exit()

try:
    with open(os.path.join(model_save_path, "class_indices.pickle"), "rb") as f:
        class_indices = pickle.load(f)
        print("Class indices:", class_indices)
except Exception as e:
    print(f"[WARNING] Couldn't load class indices: {e}")
    class_indices = {0: "with_mask", 1: "without_mask"}
    print("Using default class indices:", class_indices)

IMG_SIZE = 64

Using Haar Cascade face detector: E:\Machine Learning\dataset\haarcascade_frontalface_default.xml
[INFO] Loading face mask detector model...
Model loaded successfully
Model signature: ['dense_3']
Class indices: {0: 'with_mask', 1: 'without_mask'}


In [14]:
# Part 7: Detect and Classify Faces Function

def detect_and_classify_faces(frame):
    output_frame = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        face_roi = cv2.resize(face_roi, (IMG_SIZE, IMG_SIZE))
        face_roi = face_roi / 255.0
        face_roi = face_roi.astype(np.float32)
        face_roi = np.expand_dims(face_roi, axis=0)
        try:
            input_tensor = tf.convert_to_tensor(face_roi) #Corrected line
            prediction = infer(input_tensor)
            output_key = list(prediction.keys())[0]
            pred_values = prediction[output_key].numpy()[0]
            mask_index = np.argmax(pred_values)
            confidence = pred_values[mask_index] * 100
            if class_indices[mask_index] == "with_mask":
                label = "Mask"
                color = (0, 255, 0)
            else:
                label = "No Mask"
                color = (0, 0, 255)
            label = f"{label}: {confidence:.2f}%"
            cv2.rectangle(output_frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(output_frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        except Exception as e:
            print(f"Error during prediction: {e}")
    return output_frame

In [15]:
# Start webcam and display results
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    output_frame = detect_and_classify_faces(frame)
    cv2.imshow("Face Mask Detection", output_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# **Face Mask Detection Using CNN and OpenCV**

## **Objective**
The objective of this project is to develop an **AI-based Face Mask Detector** that can classify whether a person is wearing a mask or not in real-time using a webcam. The system utilizes **Convolutional Neural Networks (CNNs)** for image classification and **OpenCV's Haar Cascade** for face detection.

## **Steps Involved**
### **1. Data Collection and Preprocessing**
- Collected a dataset of images with and without face masks.  
- Applied **image augmentation** techniques such as rotation, zooming, and flipping to improve model generalization.  
- Rescaled images to **(64x64 pixels)** and normalized pixel values to a range of [0,1].  

### **2. Model Building**
- Built a **CNN model** using TensorFlow/Keras with:  
  - **3 convolutional layers** with ReLU activation.  
  - **Max-pooling layers** to reduce spatial dimensions.  
  - **Dropout layer** to prevent overfitting.  
  - **Fully connected layers** with softmax activation for classification.  
- Compiled the model with **Adam optimizer** and **categorical cross-entropy loss function**.  

### **3. Model Training**
- Trained the model on **5,960 images** using **10 epochs** with a batch size of **8**.  
- Achieved a final **training accuracy of 93.05%** and a **validation accuracy of 94.49%**.  
- Saved the trained model and class indices for future inference.  

### **4. Real-Time Face Mask Detection**
- Loaded the trained model and class mapping.  
- Used **OpenCV's Haar Cascade** to detect faces in webcam video frames.  
- Preprocessed detected faces and fed them into the model for classification.  
- Displayed real-time predictions on the video stream with bounding boxes and confidence scores.  

## **Conclusion**
- The developed model successfully detects **masked and unmasked faces** with high accuracy in real-time.  
- The approach can be extended to **edge devices** like **Raspberry Pi, Jetson Nano**, or **mobile applications** for practical deployment.  
- Future improvements can include **multi-class classification** (e.g., incorrectly worn masks) and **more robust face detection techniques** using deep learning-based detectors like **MTCNN or YOLO**.  
